In [ ]:
import pickle

import lightgbm
from lightgbm import LGBMRegressor
from sklearn.linear_model import LogisticRegression

from lightgbm import LGBMClassifier
import csv
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import KFold, RandomizedSearchCV
import xgboost as xgb
# data normalization with sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/fifa-common/common_data.csv', index_col = False)
cdf = pd.read_csv('/kaggle/input/original-data/data.csv', index_col = False)


In [ ]:
#y = df['Name'].values
cols = ['Club', 'Nationality', 'Name', 'Jersey Number', 'Position', 'Joined', 'Wage' , 'Release Clause',
        'Loaned From', 'Contract Valid Until', 'Club_overall', 'nation_rating', 'Overall', 'Value', 'Preferred Foot']
x = df.copy()
x.drop(cols, inplace=True, axis=1)

#x = x.values

In [ ]:
scaled = MinMaxScaler()
X = scaled.fit_transform(x)
X

In [ ]:

recommendations = NearestNeighbors(n_neighbors=6,algorithm='kd_tree')
recommendations.fit(X)
player_index = recommendations.kneighbors(X)[1]

In [ ]:
#player_index[389][1:]
for i in player_index[140][1:]:
    print (df.iloc[i]['Name'])

In [ ]:
def get_index(name):
    return df[df['Name']==name].index.tolist()[0]

In [ ]:
prefered_foot_map = {0 : 'Left', 1 : 'Right'}
def player_details (i):
    name = df.iloc[i]['Name']
    cdf_i = cdf[cdf['Name']==name].index.tolist()[0]
    print (name, "/Pos.:", cdf.iloc[cdf_i]['Position'],
           "/Age:", df.iloc[i]['Age'],
           "/Club:", df.iloc[i]['Club'],
           "/Foot:", prefered_foot_map[df.iloc[i]['Preferred Foot']],
     "/Overall:", df.iloc[i]['Overall'],
     "/Nat.:", df.iloc[i]['Nationality'],
     "/Val.:", df.iloc[i]['Value'])
    print ("\n")

In [ ]:
def recommend_similar(player):
    print ('finding recommendation for player: ')
    index=  get_index(player)
    player_details(index)
    print("These are 5 players similar to {} : ".format(player))
    
    for i in player_index[index][1:]:
        player_details(i)
        

In [ ]:
ls = get_index('L. Messi')
ls

In [ ]:
recommend_similar('Cristiano Ronaldo')

In [ ]:
pca = PCA(n_components = 2) # 2D PCA for the plot
reduced = pd.DataFrame(pca.fit_transform(X))

In [ ]:
#kmeans = KMeans(init="random",n_clusters=5,n_init=10, max_iter=300,random_state=42)
kmeans = KMeans(n_clusters=6)
kmeans.fit(reduced)
# get the cluster labels
labels = kmeans.predict(reduced)

# centroid values
centroid = kmeans.cluster_centers_

# cluster values
clusters = kmeans.labels_.tolist()

In [ ]:
reduced['cluster'] = clusters
reduced['name'] = df['Name']
reduced.columns = ['x', 'y', 'cluster', 'name']
reduced.head()

In [ ]:
from sklearn.utils import shuffle
#reduced = shuffle(reduced)
rr = reduced[0:100]
rr

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
sns.set(style="white")

ax = sns.lmplot(x="x", y="y", hue='cluster', data = reduced, legend=False,
                   fit_reg=False, size = 20, scatter_kws={"s": 250})

texts = []
for x, y, s in zip(rr.x, rr.y, rr.name):
    texts.append(plt.text(x, y, s))

ax.set(ylim=(-2, 2))
plt.tick_params(labelsize=5)
plt.xlabel("PC 1", fontsize = 5)
plt.ylabel("PC 2", fontsize = 5)

plt.show()